# Forcing uncertainties

based on `notebooks/140_WG3_draw_fair_samples.ipynb` of the AR6 WGI Chapter 7 code

Output
- `dataout/fair_samples_forcing.nc`
    Uncertainty factors and coefficients of categorized forcing agents

In [1]:
import json
import numpy as np
import pandas as pd
import scipy.stats as stats
from netCDF4 import Dataset

In [2]:
from src.util import RetrieveGitHub, retrieve_url, dffilter, write_nc

In [3]:
owner = 'IPCC-WG1'
repo = 'Chapter-7'
repo_ch7 = RetrieveGitHub(owner, repo, './datain')

In [4]:
path = repo_ch7.retrieve('data_input/random_seeds.json')
with path.open() as f1:
    SEEDS = json.load(f1)

[2024-07-05 17:44:33 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_input/random_seeds.json retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_input/random_seeds.json on 2024-06-12


In [5]:
NINETY_TO_ONESIGMA = stats.norm.ppf(0.95)

SAMPLES = 1000000
F2XCO2_MEAN = 4.00
F2XCO2_NINETY = 0.48

In [6]:
path = repo_ch7.retrieve('data_input/tunings/cmip6_aerosol_coefficients.json')
with path.open() as f1:
    cmip6_aerosol_data = json.load(f1)

[2024-07-05 17:44:33 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_input/tunings/cmip6_aerosol_coefficients.json retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_input/tunings/cmip6_aerosol_coefficients.json on 2024-06-12


## Scaling factors of categorized forcings

In [7]:
# 5-95% uncertainty ranges
unc_ranges = np.array([
    0.12,      # CO2
    0.20,      # CH4: updated value from etminan 2016
    0.14,      # N2O
    0.19,      # other WMGHGs
    0.50,      # Total ozone
    1.00,      # stratospheric WV from CH4
    0.70,      # contrails approx - half-normal
    1.25,      # bc on snow - half-normal
    0.50,      # land use change
    5.0/20.0,  # volcanic
    0.50,      # solar (amplitude)
]) / NINETY_TO_ONESIGMA

NORMALS = len(unc_ranges)

# standard deviations of the scale factor for normally distributed forcings
scale_normals = stats.norm.rvs(
    size=(SAMPLES, NORMALS),
    loc=np.ones((SAMPLES, NORMALS)),
    scale=np.ones((SAMPLES, NORMALS)) * unc_ranges[None, :],
    random_state=SEEDS[4]
)

# Apply asymmetric Gaussian to bc on snow and contrails
# by scaling the half of the distribution above/below best estimate
bl = scale_normals[:, 7] < 1
scale_normals[bl, 7] = 0.08 / 0.1 * (scale_normals[bl, 7] - 1) + 1
bl = scale_normals[:, 6] < 1
scale_normals[bl, 6] = 0.0384 / 0.0406 * (scale_normals[bl, 6] - 1) + 1

trend_solar = stats.norm.rvs(
    size=SAMPLES,
    loc=+0.01,
    scale=0.07/NINETY_TO_ONESIGMA,
    random_state=SEEDS[50],
)

## Aerosol uncertainties

In [8]:
models = [
    'CanESM5',
    'E3SM',
    'GFDL-ESM4',
    'GFDL-CM4',
    'GISS-E2-1-G',
    'HadGEM3-GC31-LL',
    'IPSL-CM6A-LR',
    'MIROC6',
    'MRI-ESM2-0',
    'NorESM2-LM',
    'UKESM1-0-LL',
]
cmip6_aci = np.array([
    [
        cmip6_aerosol_data[model]['ERFaci'][species]
        for species in ['n0', 'n1']
    ]
    for model in models
])
cmip6_aci = np.log(cmip6_aci)

In [9]:
kde = stats.gaussian_kde(cmip6_aci.T)
aci_coeffs = np.exp(kde.resample(size=int(SAMPLES), seed=SEEDS[8]).T)

In [10]:
# target ranges for aerosols:
# total ERFari -0.6 -0.3 -0.0
## BC 0.05 0.4 0.8 then subtract -0.1 for RA so -0.05 0.3 0.7
## SO2 -0.6 -0.4 -0.2
## OC -0.16 -0.09 -0.03
## Nitrate -0.3 -0.11 -0.03
bc_20101750 = stats.norm.rvs(
    loc=0.3, scale=0.2/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[95],
)
oc_20101750 = stats.norm.rvs(
    loc=-0.09, scale=0.07/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[96],
)
so2_20101750 = stats.norm.rvs(
    loc=-0.4, scale=0.2/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[97],
)
nit_20101750 = stats.norm.rvs(
    loc=-0.11, scale=0.05/NINETY_TO_ONESIGMA, size=SAMPLES, random_state=SEEDS[98],
)

In [11]:
np.percentile(bc_20101750+oc_20101750+so2_20101750+nit_20101750, (5, 50, 95))

array([-0.59508118, -0.29988114, -0.00482923])

In [12]:
fn = 'rcmip-emissions-annual-means-v5-1-0.csv'
path = retrieve_url(
    f'./datain/rcmip/{fn}',
    f'https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/{fn}',
)

[2024-07-05 17:44:35 src.util] INFO:Use local file datain/rcmip/rcmip-emissions-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-emissions-annual-means-v5-1-0.csv on 2024-05-25


In [13]:
# Get SSP historical emissions
df_emis_rcmip = pd.read_csv(path, index_col=list(range(7))).rename(columns=int)

In [14]:
kw = {'Scenario': 'ssp245', 'Region': 'World'}
species = [
    'Sulfur', 'BC', 'OC', 'NH3', 'NOx',
]
df = dffilter(df_emis_rcmip, Variable=[f'Emissions|{x}' for x in species], **kw)
df

1750  \
Model           Scenario Region Variable         Unit      Mip_Era Activity_Id                 
MESSAGE-GLOBIOM ssp245   World  Emissions|BC     Mt BC/yr  CMIP6   not_applicable   2.097771   
                                Emissions|NH3    Mt NH3/yr CMIP6   not_applicable   6.927690   
                                Emissions|NOx    Mt NOx/yr CMIP6   not_applicable  12.735212   
                                Emissions|OC     Mt OC/yr  CMIP6   not_applicable  15.447668   
                                Emissions|Sulfur Mt SO2/yr CMIP6   not_applicable   2.440048   

                                                                                        1751  \
Model           Scenario Region Variable         Unit      Mip_Era Activity_Id                 
MESSAGE-GLOBIOM ssp245   World  Emissions|BC     Mt BC/yr  CMIP6   not_applicable   2.072926   
                                Emissions|NH3    Mt NH3/yr CMIP6   not_applicable   6.860142   
                                Emissions|NOx    Mt NOx/yr CMIP6   not_applicable  12.592303   
                                Emissions|OC     Mt OC/yr  CMIP6   not_applicable  15.188717   
                                Emissions|Sulfur Mt SO2/yr CMIP6   not_applicable   2.408379   

                                                                                        1752  \
Model           Scenario Region Variable         Unit      Mip_Era Activity_Id                 
MESSAGE-GLOBIOM ssp245   World  Emissions|BC     Mt BC/yr  CMIP6   not_applicable   2.064312   
                                Emissions|NH3    Mt NH3/yr CMIP6   not_applicable   6.819482   
                                Emissions|NOx    Mt NOx/yr CMIP6   not_applicable  12.594270   
                                Emissions|OC     Mt OC/yr  CMIP6   not_applicable  15.034476   
                                Emissions|Sulfur Mt SO2/yr CMIP6   not_applicable   2.397116   

                                                                                        1753  \
Model           Scenario Region Variable         Unit      Mip_Era Activity_Id                 
MESSAGE-GLOBIOM ssp245   World  Emissions|BC     Mt BC/yr  CMIP6   not_applicable   2.071951   
                                Emissions|NH3    Mt NH3/yr CMIP6   not_applicable   6.881439   
                                Emissions|NOx    Mt NOx/yr CMIP6   not_applicable  12.539820   
                                Emissions|OC     Mt OC/yr  CMIP6   not_applicable  15.182545   
                                Emissions|Sulfur Mt SO2/yr CMIP6   not_applicable   2.410693   

                                                                                        1754  \
Model           Scenario Region Variable         Unit      Mip_Era Activity_Id                 
MESSAGE-GLOBIOM ssp245   World  Emissions|BC     Mt BC/yr  CMIP6   not_applicable   2.099350   
                                Emissions|NH3    Mt NH3/yr CMIP6   not_applicable   7.015189   
                                Emissions|NOx    Mt NOx/yr CMIP6   not_applicable  12.636837   
                                Emissions|OC     Mt OC/yr  CMIP6   not_applicable  15.558773   
                                Emissions|Sulfur Mt SO2/yr CMIP6   not_applicable   2.462065   

                                                                                        1755  \
Model           Scenario Region Variable         Unit      Mip_Era Activity_Id                 
MESSAGE-GLOBIOM ssp245   World  Emissions|BC     Mt BC/yr  CMIP6   not_applicable   2.099173   
                                Emissions|NH3    Mt NH3/yr CMIP6   not_applicable   6.943989   
                                Emissions|NOx    Mt NOx/yr CMIP6   not_applicable  12.765991   
                                Emissions|OC     Mt OC/yr  CMIP6   not_applicable  15.358183   
                                Emissions|Sulfur Mt SO2/yr CMIP6   not_applicable   2.442355   

                                                  

In [15]:
df = (
    df
    .loc[:, :2100]
    .droplevel([x for x in df.index.names if x!='Variable'])
    .rename(lambda x: x.split('|')[1])
    .rename({'Sulfur': 'SO2'})
    .interpolate(axis=1)
)
df

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
Variable,,,,,,,,,,,,,,,,,,,,,
BC,2.097771,2.072926,2.064312,2.071951,2.099350,2.099173,2.068915,2.147924,2.120611,2.117267,...,2.986676,2.949374,2.912072,2.874770,2.837468,2.800165,2.762863,2.725561,2.688259,2.650957
NH3,6.927690,6.860142,6.819482,6.881439,7.015189,6.943989,6.922712,7.047674,6.995460,7.045958,...,67.182686,66.981229,66.779773,66.578316,66.376860,66.175403,65.973947,65.772490,65.571033,65.369577
NOx,12.735212,12.592303,12.594270,12.539820,12.636837,12.765991,12.416163,13.133152,12.979345,12.752546,...,82.505487,81.970669,81.435851,80.901033,80.366215,79.831397,79.296580,78.761762,78.226944,77.692126
OC,15.447668,15.188717,15.034476,15.182545,15.558773,15.358183,15.195777,15.704819,15.514612,15.574013,...,15.579334,15.463816,15.348297,15.232778,15.117259,15.001741,14.886222,14.770703,14.655184,14.539666
SO2,2.440048,2.408379,2.397116,2.410693,2.462065,2.442355,2.419184,2.507465,2.479744,2.473528,...,32.927025,32.693177,32.459329,32.225481,31.991633,31.757785,31.523938,31.290090,31.056242,30.822394


In [16]:
map_conversion = {
    'SO2': 32./64.,
    'NOx': 14./46.,
}
for k, v in map_conversion.items():
    df.loc[k] *= v

df

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
Variable,,,,,,,,,,,,,,,,,,,,,
BC,2.097771,2.072926,2.064312,2.071951,2.099350,2.099173,2.068915,2.147924,2.120611,2.117267,...,2.986676,2.949374,2.912072,2.874770,2.837468,2.800165,2.762863,2.725561,2.688259,2.650957
NH3,6.927690,6.860142,6.819482,6.881439,7.015189,6.943989,6.922712,7.047674,6.995460,7.045958,...,67.182686,66.981229,66.779773,66.578316,66.376860,66.175403,65.973947,65.772490,65.571033,65.369577
NOx,3.875934,3.832440,3.833039,3.816467,3.845994,3.885302,3.778832,3.997046,3.950236,3.881210,...,25.110366,24.947595,24.784824,24.622054,24.459283,24.296512,24.133742,23.970971,23.808200,23.645430
OC,15.447668,15.188717,15.034476,15.182545,15.558773,15.358183,15.195777,15.704819,15.514612,15.574013,...,15.579334,15.463816,15.348297,15.232778,15.117259,15.001741,14.886222,14.770703,14.655184,14.539666
SO2,1.220024,1.204190,1.198558,1.205347,1.231033,1.221178,1.209592,1.253732,1.239872,1.236764,...,16.463512,16.346589,16.229665,16.112741,15.995817,15.878893,15.761969,15.645045,15.528121,15.411197


In [17]:
df_2010_1750 = df.loc[:, 2005:2014].mean(axis=1) - df.loc[:, 1750]

In [18]:
beta_bc = bc_20101750 / df_2010_1750.loc['BC']
beta_oc = oc_20101750 / df_2010_1750.loc['OC']
beta_so2 = so2_20101750 / df_2010_1750.loc['SO2']
beta_nh3 = nit_20101750 / df_2010_1750.loc['NH3']

In [19]:
dfa = df.sub(df.loc[:, 1750], axis=0)

ERFari_2010 = (
    dfa.loc['SO2', 2010] * beta_so2
    + dfa.loc['BC', 2010] * beta_bc
    + dfa.loc['OC', 2010] * beta_oc
    + dfa.loc['NH3', 2010] * beta_nh3
)

In [20]:
ERFaci_scale = stats.norm.rvs(
    size=SAMPLES, loc=-1.0, scale=0.7/NINETY_TO_ONESIGMA, random_state=SEEDS[9],
)

In [21]:
np.percentile(ERFaci_scale, (5, 16, 50, 84, 95))

array([-1.69913916, -1.42248834, -1.00051153, -0.57760861, -0.30205032])

In [22]:
def ghan(x, beta, n0, n1):
    """ERFaci logarithmic in emissions excluding nitrate.

    Named after Steve Ghan, whose 2013 simple emissions emulator is extremely useful,
    (https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/jgrd.50567),
    and can be emulated again using this very simple formula.

    Inputs
    ------
    x : obj:`numpy.array`
        Time series of aerosol emissions
    beta : float
        Scale factor linking forcing to time series
    n0 : float
        Shape factor for SO2 emissions, W m**-2 (TgSO2 yr**-1)**-1
    n1 : float
        Shape factor for BC+OC emissions, W m**-2 (TgC yr**-1)**-1

    Returns
    -------
    res : obj:`numpy.array`
        Time series of ERFaci
    """
    return -beta*np.log(1 + x[0]/n0 + x[1]/n1)

In [23]:
forcing2010 = ghan(
    [
        df.loc['SO2', 2005:2014].values.reshape((-1, 1)),
        (df.loc['BC', 2005:2014] + df.loc['OC', 2005:2014]).values.reshape((-1, 1)),
    ], 0.97, aci_coeffs[:, 0], aci_coeffs[:, 1],
).mean(axis=0)

forcing1750 = ghan(
    [df.loc['SO2', 1750], df.loc['BC', 1750] + df.loc['OC', 1750]],
    0.97, aci_coeffs[:, 0], aci_coeffs[:, 1],
)

beta = ERFaci_scale / (forcing2010 - forcing1750)

In [24]:
ERFaci_2010 = (
    ghan(
        [df.loc['SO2', 2010], df.loc['BC', 2010] + df.loc['OC', 2010]],
        0.97, aci_coeffs[:, 0], aci_coeffs[:, 1],
    )
    - forcing1750
) * beta

In [25]:
np.percentile(
    ERFari_2010 + ERFaci_2010,
    (5, 16, 50, 84, 95),
)

array([-2.05711085, -1.75399445, -1.29033147, -0.82696466, -0.52477934])

## Save data

In [26]:
dims = ('Member',)
var_dict = {'Member': (np.arange(1, dtype='i8'), dims, {})}
var_dict.update({
    f'scale_normals__{agent}': (scale_normals[:1, i], dims, {})
    for i, agent in enumerate([
        'co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
        'contrails', 'bc_on_snow', 'land_use', 'volcanic', 'solar',
    ])
})
var_dict.update({
    f'aci_coeffs__{i}': (aci_coeffs[:1, i], dims, {})
    for i in range(aci_coeffs.shape[1])
})
var_dict.update({
    x: (globals()[x][:1], dims, {})
    for x in [
        'trend_solar',
        'beta_so2', 'beta_bc', 'beta_oc', 'beta_nh3', 'beta',
    ]
})

In [27]:
path_out = './dataout/fair_samples_forcing.nc'
write_nc(path_out, var_dict, {}, dim_unlimited='Member')

In [28]:
ncf = Dataset(path_out, 'r+')

ncf.variables['Member'][:] = np.arange(SAMPLES, dtype='i8')

for i, agent in enumerate([
    'co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
    'contrails', 'bc_on_snow', 'land_use', 'volcanic', 'solar',
]):
    ncf.variables[f'scale_normals__{agent}'][:] = scale_normals[:, i]

for i in range(aci_coeffs.shape[1]):
    ncf.variables[f'aci_coeffs__{i}'][:] = aci_coeffs[:, i]

for x in [
    'trend_solar',
    'beta_so2', 'beta_bc', 'beta_oc', 'beta_nh3', 'beta',
]:
    ncf.variables[x][:] = globals()[x][:]

ncf.close()